# **Thermodynamic Corrections Notebook**

This notebook provides small helper functions to compute molecular
thermodynamic corrections from quantum-chemical frequencies:

- Zero-Point Energy (ZPE) Corrections
- Vibrational enthalpy and entropy corrections
- Translational enthalpy and entropy corrections (ideal gas)

All energies are per mole in J/mol (entropy in J/mol/K).
Frequencies are **harmonic frequencies in cm⁻¹**.
Mass is **molar mass in g/mol**.


In [19]:
import numpy as np

# Physical constants (CODATA 2018)
kB = 1.380649e-23        # Boltzmann constant, J/K
h  = 6.62607015e-34      # Planck constant, J*s
c  = 299792458.0         # speed of light, m/s
NA = 6.02214076e23       # Avogadro's number, 1/mol

R  = NA * kB             # gas constant, J/mol/K


## **1. Zero-Point Energy (ZPE)**

For a set of harmonic vibrational frequencies with characteristic temperatures:

$$
E_{\mathrm{ZPE}}
= \frac{1}{2} \sum_i h\nu_i
= \frac{1}{2} R \sum_i \theta_i
$$

In [20]:
def cm1_to_theta(freq_cm):
    """
    Convert a vibrational wavenumber (cm^-1) to the characteristic
    vibrational temperature θ_v = h c ν / k_B (in K).

    freq_cm : float or array, in cm^-1
    """
    freq_cm = np.array(freq_cm, dtype=float)

    # convert cm^-1 -> m^-1 (× 100), then θ = h c ν / kB
    return (h * c * freq_cm * 100.0) / kB

def ZeroPointEnergy(freq_cm):
    freq_cm = np.array(freq_cm, dtype=float)
    theta = cm1_to_theta(freq_cm)          # characteristic temperatures θ_i
    return 0.5 * R * np.sum(theta)

ZPE Examples:

In [21]:
H2_Vibs = [4342]

print(f"H2 Gas - ZPE: {ZeroPointEnergy(H2_Vibs)/1000:.{4}g} kJ/mol, Vibs: {H2_Vibs} cm-1")

H2O_Vibs = [3585, 3506, 1885]

print(f"H2O Gas - ZPE: {ZeroPointEnergy(H2O_Vibs)/1000:.{4}g} kJ/mol, Vibs: {H2O_Vibs} cm-1")

CO_Vibs = [2143]
print(f"CO Gas - ZPE: {ZeroPointEnergy(CO_Vibs)/1000:.{4}g} kJ/mol, Vibs: {CO_Vibs} cm-1")

CO_ads_FE100 = [1189.6, 341.0, 328.4, 294.7, 203.9, 131.9]
print(f"CO Adsorbed on Fe(100) - ZPE: {ZeroPointEnergy(CO_ads_FE100)/1000:.{4}g} kJ/mol, Vibs: {CO_ads_FE100} cm-1")


CO2_Vibs = [2565, 1480, 526, 526]
print(f"CO2 Gas - ZPE: {ZeroPointEnergy(CO2_Vibs)/1000:.{4}g} kJ/mol, Vibs: {CO2_Vibs} cm-1")

Benzene_Vibs = [3206, 3195, 3195, 
                3187, 3187, 3184, 
                1767, 1767, 1579, 
                1579, 1366, 1329, 
                1276, 1222, 1222, 
                1179, 1146, 1146, 
                1028, 1012, 989, 
                989, 891, 891, 
                744, 648, 648, 
                618, 371, 371]
print(f"Benzene Gas - ZPE: {ZeroPointEnergy(Benzene_Vibs)/1000:.{4}g} kJ/mol, Vibs: {Benzene_Vibs} cm-1")




H2 Gas - ZPE: 25.97 kJ/mol, Vibs: [4342] cm-1
H2O Gas - ZPE: 53.69 kJ/mol, Vibs: [3585, 3506, 1885] cm-1
CO Gas - ZPE: 12.82 kJ/mol, Vibs: [2143] cm-1
CO Adsorbed on Fe(100) - ZPE: 14.89 kJ/mol, Vibs: [1189.6, 341.0, 328.4, 294.7, 203.9, 131.9] cm-1
CO2 Gas - ZPE: 30.49 kJ/mol, Vibs: [2565, 1480, 526, 526] cm-1
Benzene Gas - ZPE: 268.8 kJ/mol, Vibs: [3206, 3195, 3195, 3187, 3187, 3184, 1767, 1767, 1579, 1579, 1366, 1329, 1276, 1222, 1222, 1179, 1146, 1146, 1028, 1012, 989, 989, 891, 891, 744, 648, 648, 618, 371, 371] cm-1


# **2. Vibrational Enthalpy and Entropy (Harmonic Oscillator)**

### **Characteristic vibrational temperature**

$$
\theta_i = \frac{h\nu_i}{k_B}
$$

### **Total vibrational enthalpy (including ZPE)**

$$
H_{\mathrm{vib}}(T)
= R \sum_i \theta_i
\left(
\frac{1}{2} +
\frac{1}{e^{\theta_i/T} - 1}
\right)
$$

### **Thermal vibrational enthalpy (excluding ZPE)**

$$
\Delta H_{\mathrm{vib}}^{\mathrm{th}}(T)
= R \sum_i \theta_i
\left(
\frac{1}{e^{\theta_i/T} - 1}
\right)
$$

### **Vibrational entropy**

$$
S_{\mathrm{vib}}(T)
= R \sum_i
\left[
\frac{\theta_i/T}{e^{\theta_i/T} - 1}

\ln
\left(
1 - e^{-\theta_i/T}
\right)
\right]
$$

In [22]:
def vibrational_thermo(freqs_cm, T):
    """
    Vibrational thermodynamic functions from harmonic frequencies.

    Parameters
    ----------
    freqs_cm : array-like
        Vibrational frequencies in cm^-1 (positive, non-imaginary modes).
    T : float
        Temperature in K.

    Returns
    -------
    dict with keys:
        ZPE         : Zero-point energy, J/mol
        H_vib_total : Total vibrational enthalpy incl. ZPE, J/mol
        H_vib_th    : Thermal vibrational enthalpy (H(T)-ZPE), J/mol
        S_vib       : Vibrational entropy, J/mol/K
    """
    freqs_cm = np.array(freqs_cm, dtype=float)

    # filter out zero/negative modes if present (e.g. translations, rotations, TS mode)
    freqs_cm = freqs_cm[freqs_cm > 0.0]
    if freqs_cm.size == 0:
        return dict(ZPE=0.0, H_vib_total=0.0, H_vib_th=0.0, S_vib=0.0)

    theta = cm1_to_theta(freqs_cm)          # characteristic temperatures θ_i

    x = theta / T
    exp_x = np.exp(x)

    # Zero-point energy per mode: 0.5 * R * θ
    ZPE = 0.5 * R * np.sum(theta)

    # Total vibrational enthalpy incl. ZPE
    H_vib_total = R * np.sum(theta * (0.5 + 1.0 / (exp_x - 1.0)))

    # Thermal part only (above ZPE)
    H_vib_th = R * np.sum(theta * (1.0 / (exp_x - 1.0)))

    # Vibrational entropy
    S_vib = R * np.sum(x / (exp_x - 1.0) - np.log(1.0 - np.exp(-x)))

    return {
        "ZPE": ZPE,
        "H_vib_total": H_vib_total,
        "H_vib_th": H_vib_th,
        "S_vib": S_vib,
    }

# Quick test example
T_test = 298.15
freqs_test = [1000.0, 1500.0, 3000.0]  # cm^-1
vib = vibrational_thermo(freqs_test, T_test)
print("ZPE (kJ/mol)       :", vib["ZPE"] / 1000)
print("H_vib_th (kJ/mol)  :", vib["H_vib_th"] / 1000)
print("S_vib (J/mol/K)    :", vib["S_vib"])


ZPE (kJ/mol)       : 32.89730555064167
H_vib_th (kJ/mol)  : 0.10964764480450444
S_vib (J/mol/K)    : 0.4406992601364041


# **3. Translational Enthalpy & Entropy (Ideal Gas)**

### **Translational enthalpy**

Total translational enthalpy for 1 mole:

$$
H_{\mathrm{trans}}(T)
= \frac{5}{2} RT
$$

### **Translational entropy (Sackur–Tetrode equation)**

For a particle of mass ( $$m = \frac{M}{N_A}$$ ), pressure ( p ):

$$
S_{\mathrm{trans}}(T,p)
= R \left[
\ln \left(
\left(
\frac{2\pi m k_B T}{h^2}
\right)^{3/2}
\frac{k_B T}{P}
\right)

* \frac{5}{2}
  \right]
  $$

In [23]:
def translational_thermo(molar_mass_g_mol, T, p):
    """
    Translational contributions for an ideal gas (1 mol).

    Parameters
    ----------
    molar_mass_g_mol : float
        Molar mass in g/mol.
    T : float
        Temperature in K.
    p : float
        Pressure in Pa.

    Returns
    -------
    dict with keys:
        H_trans : translational enthalpy (H(T)-H(0)), J/mol
        S_trans : translational entropy, J/mol/K
    """
    # convert molar mass (g/mol) -> mass per molecule (kg)
    M = molar_mass_g_mol / 1000.0      # kg/mol
    m = M / NA                         # kg per molecule

    # Enthalpy of translation (relative to 0 K)
    H_trans = 2.5 * R * T  # 5/2 RT

    # Entropy of translation (Sackur-Tetrode, 1 mol)
    prefactor = ((2 * np.pi * m * kB * T) / (h**2))**1.5 * (kB * T / p)
    S_trans = R * (np.log(prefactor) + 2.5)

    return {
        "H_trans": H_trans,
        "S_trans": S_trans,
    }

# Quick test for N2 at 298 K, 1 bar
T_test = 298.15
p_test = 1.0e5   # Pa ~ 1 bar
thermo_N2 = translational_thermo(molar_mass_g_mol=28.0, T=T_test, p=p_test)
print("H_trans (kJ/mol):", thermo_N2["H_trans"]/1000)
print("S_trans (J/mol/K):", thermo_N2["S_trans"])


H_trans (kJ/mol): 6.197392574005971
S_trans (J/mol/K): 150.4135427565377


# **4. Rotational Enthalpy & Entropy**

## **(a) Linear Molecules**

For rotational constant ( B ) (in cm-1), symmetry number ( $$\sigma$$ ):

### **Rotational temperature**

$$
\theta_r = \frac{h c B}{k_B}
$$

### **Rotational enthalpy**

$$
H_{\mathrm{rot}}(T)
= RT
$$

### **Rotational entropy**

$$
S_{\mathrm{rot}}(T)
= R \left[
\ln\left(
\frac{T}{\sigma, \theta_r}
\right)

* 1
  \right]
  $$



## **(b) Non-Linear Molecules**

Rotational constants ( A, B, C ), with rotational temperatures ( $$\theta_A, \theta_B, \theta_C$$ ):

### **Rotational enthalpy**

$$
H_{\mathrm{rot}}(T)
= \frac{3}{2} RT
$$

### **Rotational entropy**

$$
S_{\mathrm{rot}}(T)
= R \left[
\ln\left(
\frac{\sqrt{\pi}, T^{3/2}}{\sigma, \sqrt{\theta_A \theta_B \theta_C}}
\right)

* \frac{3}{2}
  \right]
  $$

In [24]:
# We work with rotational constants in cm^-1 and symmetry number σ.
# θ_r (or θ_A, θ_B, θ_C) are the characteristic rotational temperatures:
#   θ = h c B / k_B
# where B is a rotational constant in cm^-1.

def cm1_to_theta_rot(B_cm):
    """Convert rotational constant(s) in cm^-1 to rotational temperature(s) θ in K."""
    B_cm = np.array(B_cm, dtype=float)
    return (h * c * B_cm * 100.0) / kB  # cm^-1 -> m^-1, then θ = h c ν / kB


def rotational_thermo_linear(B_cm, sigma, T):
    """
    Rotational enthalpy & entropy for a *linear* molecule (rigid rotor).

    Parameters
    ----------
    B_cm : float
        Rotational constant in cm^-1 (for a linear rotor).
    sigma : int
        Rotational symmetry number (e.g. 2 for homonuclear diatomics like N2, O2).
    T : float
        Temperature in K.

    Returns
    -------
    dict with keys:
        H_rot : J/mol   (H(T) - H(0))
        S_rot : J/mol/K
    """
    theta_r = cm1_to_theta_rot(B_cm)  # single value

    # High-temperature rigid-rotor limit:
    # U_rot = RT      -> H_rot = RT
    H_rot = R * T

    # Sackur-like expression for rotational entropy of a linear rotor:
    # S_rot = R [ ln(T / (sigma * θ_r)) + 1 ]
    S_rot = R * (np.log(T / (sigma * theta_r)) + 1.0)

    return {
        "H_rot": H_rot,
        "S_rot": S_rot,
    }


def rotational_thermo_nonlinear(Bs_cm, sigma, T):
    """
    Rotational enthalpy & entropy for a *non-linear* molecule (asymmetric top).

    Parameters
    ----------
    Bs_cm : array-like of length 3
        Rotational constants (A, B, C) in cm^-1.
    sigma : int
        Rotational symmetry number.
    T : float
        Temperature in K.

    Returns
    -------
    dict with keys:
        H_rot : J/mol   (H(T) - H(0))
        S_rot : J/mol/K
    """
    Bs_cm = np.array(Bs_cm, dtype=float)
    if Bs_cm.size != 3:
        raise ValueError("Non-linear rotor needs 3 rotational constants (A, B, C).")

    theta = cm1_to_theta_rot(Bs_cm)  # θ_A, θ_B, θ_C
    theta_A, theta_B, theta_C = theta

    # High-temperature rigid-rotor limit:
    # U_rot = (3/2) RT   ->  H_rot = (3/2) RT
    H_rot = 1.5 * R * T

    # Rotational entropy for a non-linear rotor (asymmetric top):
    # q_rot = sqrt(pi) * T^(3/2) / (sigma * sqrt(θ_A θ_B θ_C))
    # S_rot = R [ ln(q_rot) + 3/2 ]
    prefactor = np.sqrt(np.pi) * T**1.5 / (sigma * np.sqrt(theta_A * theta_B * theta_C))
    S_rot = R * (np.log(prefactor) + 1.5)

    return {
        "H_rot": H_rot,
        "S_rot": S_rot,
    }

T_test = 298.15

# 1) Linear example: N2
# Rotational constant B ≈ 1.99 cm^-1, sigma = 2 for homonuclear diatomic
rot_N2 = rotational_thermo_linear(B_cm=1.99, sigma=2, T=T_test)
print("=== N2 (linear) at 298 K ===")
print("H_rot (kJ/mol):", rot_N2["H_rot"] / 1000)
print("S_rot (J/mol/K):", rot_N2["S_rot"])

# 2) Non-linear example: H2O
# Approx rotational constants (gas phase) A,B,C ~ 27.9, 14.5, 9.3 cm^-1 (order not critical here)
rot_H2O = rotational_thermo_nonlinear(Bs_cm=[27.9, 14.5, 9.3], sigma=2, T=T_test)
print("\n=== H2O (non-linear) at 298 K ===")
print("H_rot (kJ/mol):", rot_H2O["H_rot"] / 1000)
print("S_rot (J/mol/K):", rot_H2O["S_rot"])


=== N2 (linear) at 298 K ===
H_rot (kJ/mol): 2.478957029602388
S_rot (J/mol/K): 41.17755516206851

=== H2O (non-linear) at 298 K ===
H_rot (kJ/mol): 3.718435544403582
S_rot (J/mol/K): 43.76340615928339
